In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.linalg import norm
import flopy
import flopy.utils.binaryfile as bf
%matplotlib inline
%config InlineBackend.figure_format='retina'
import rasterio
from tqdm import tqdm
from scipy.ndimage import maximum_filter
from scipy.interpolate import griddata

In [ ]:
import sys
sys.path.append('../scripts/')
from Slate_Floodplain_MODFLOW import *

In [ ]:
ncol = 480
nrow = 460

nlay = 16
soil_nlay = 10
gravel_nlay = nlay - soil_nlay
dam_nlay = 5

# setting up the vertical discretization and model bottom elevation
zbot = np.zeros((nlay,nrow,ncol))

# Soil layers
for lay in np.arange(0,soil_nlay):    
    zbot[lay,:,:] = DEM - np.maximum(gravel_interface*((lay+1)/soil_nlay),0.1*(lay+1)) 

# Gravel layers
gravel_discretized_ratio = [0.02,0.04,0.1,0.3,0.6,1]
for i, lay in enumerate(np.arange(soil_nlay, nlay)):
    zbot[lay,:,:] = zbot[soil_nlay-1,:,:] - np.maximum(bedrock_interface*gravel_discretized_ratio[i],0.1*(i+1))
    
thickness = np.zeros(zbot.shape)
for i in range(nlay):
    if i == 0:
        thickness[i,:,:] = DEM-zbot[i,:,:]
    else:
        thickness[i,:,:] = zbot[i-1,:,:]-zbot[i,:,:]
depth_to_surface = DEM-zbot

In [ ]:
# Default values 
hk_gravel = 2e-3 #m/s
hk_soil = 1.4e-5 #m/s
vka_ratio_gravel = 0.5 #m/s
vka_ratio_soil = 0.1 #m/s
k_dam = 1e-7 #m/s

precip = 2e-3 #m/d
ET = 2e-3 #m/d

In [ ]:
mf,head,hk,vka,strt,zbot,flf,frf = modflow_BC(hk_gravel,hk_soil,vka_ratio_gravel,vka_ratio_soil,k_dam,
                                              ET,precip-ET, 'test', './baseflow_test')

# Local and Regional Groundwater Responses

In [ ]:
downstream = np.load('../data/characterization_data/Beaver_pond_dam/floodplain_downstream.npy')
upstream = np.load('../data/characterization_data/Beaver_pond_dam/floodplain_upstream.npy')
downstream[:,300:] = 0

In [ ]:
# Downward flow on top of gravel bed (river)
def dw_river(flf):
    spatial_mask = (river+pond-dam ==2)
    velocity = flf[soil_nlay-1,:,:]*spatial_mask
    return velocity

# Downward flow on top of soil layer (floodplain)
def dw_floodplain(flf):
    spatial_mask = (upstream+downstream==1)
    velocity = flf[soil_nlay-1,:,:]*spatial_mask
    return velocity

# Horizontal flow though dam cross section (river)
def h_dam_river(frf,dam_section = 240):
    velocity = (frf/thickness)[:,:,dam_section]
    line_mask = np.zeros(frf.shape[1])
    line_mask[np.where((river+pond-dam ==2))[0].min():(np.where((river+pond-dam ==2))[0].max()+2)] = 1
    spatial_mask = line_mask[np.newaxis, :]==1
    velocity = velocity*spatial_mask
    return velocity

# Horizontal flow though dam cross section (floodplain)
def h_dam_floodplain(frf,dam_section = 240):
    velocity = (frf/thickness)[:,:,dam_section]
    spatial_mask = (upstream+downstream)[:,dam_section][np.newaxis, :]==1
    velocity = velocity*spatial_mask
    return velocity

In [ ]:
def local_regional_response(flf,frf):
    dam_section = 240
    dw_r = dw_river(flf)
    dw_f = dw_floodplain(flf)
    h_v_r = h_dam_river(frf)
    h_v_f = h_dam_floodplain(frf)
    h_flux_r = h_v_r*thickness[:,:,dam_section]
    h_flux_f = h_v_f*thickness[:,:,dam_section]
    return dw_r,dw_f,h_v_r,h_v_f,h_flux_r,h_flux_f

In [ ]:
forcings = pd.read_csv('../data/response_data/preprocessing/period_forcings.csv')

# Result for Set 2, calibrated baseflow

In [ ]:
num_MC = 300

all_dw_r = np.zeros((num_MC, nrow,ncol))
all_dw_f = np.zeros((num_MC, nrow,ncol))
all_h_v_r = np.zeros((num_MC,nlay, nrow))
all_h_v_f = np.zeros((num_MC,nlay, nrow))
all_h_flux_r = np.zeros((num_MC,nlay, nrow))
all_h_flux_f = np.zeros((num_MC,nlay, nrow))
all_head = np.zeros((num_MC,nlay, nrow,ncol))

for i in tqdm(range(num_MC)):
    head,flf,frf = read_sim('./Posterior_Simulation/sim'+str(i).zfill(3),'sim'+str(i).zfill(3))
    all_head[i,:]  = head
    # Calculate responses for current case
    dw_r_i, dw_f_i, h_v_r_i, h_v_f_i, h_flux_r_i, h_flux_f_i = local_regional_response(flf, frf)

    # Store results in arrays
    all_dw_r[i, :] = dw_r_i
    all_dw_f[i, :] = dw_f_i
    all_h_v_r[i] = h_v_r_i
    all_h_v_f[i] = h_v_f_i
    all_h_flux_r[i, :] = h_flux_r_i
    all_h_flux_f[i, :] = h_flux_f_i

In [ ]:
# Assuming 'all_head' is your data array
data = np.mean(all_head, axis=0)[5, :, :]
data[data<0] = np.nan
# Plot the image
plt.imshow(data, cmap='magma', vmin=2724, vmax=2726)

# Add contour lines
contour_levels = np.linspace(2724, 2726, 10)  # Adjust the number of contour levels as needed
plt.contour(data, levels=contour_levels, colors='grey')

plt.axis('off')
#plt.colorbar()  # Add colorbar for reference


In [ ]:
spd = 60*60*24
all_dw_r_sum = np.sum((all_dw_r*(all_dw_r>0)).reshape(num_MC,-1),axis = 1)/spd
all_dw_f_sum = np.sum((all_dw_f*(all_dw_f>0)).reshape(num_MC,-1),axis = 1)/spd
all_h_flux_r_sum = np.sum((all_h_flux_r*(all_h_flux_r>0)).reshape(num_MC,-1),axis = 1)/spd
all_h_flux_f_sum = np.sum((all_h_flux_f*(all_h_flux_f>0)).reshape(num_MC,-1),axis = 1)/spd

# Result for Set 2, snowmelt

In [ ]:
num_MC = 300

all_dw_r = np.zeros((num_MC, nrow,ncol))
all_dw_f = np.zeros((num_MC, nrow,ncol))
all_h_v_r = np.zeros((num_MC,nlay, nrow))
all_h_v_f = np.zeros((num_MC,nlay, nrow))
all_h_flux_r = np.zeros((num_MC,nlay, nrow))
all_h_flux_f = np.zeros((num_MC,nlay, nrow))

for i in tqdm(range(num_MC)):
    head,flf,frf = read_sim('./Posterior_Simulation_Snowmelt/sim'+str(i).zfill(3),'sim'+str(i).zfill(3))

    # Calculate responses for current case
    dw_r_i, dw_f_i, h_v_r_i, h_v_f_i, h_flux_r_i, h_flux_f_i = local_regional_response(flf, frf)

    # Store results in arrays
    all_dw_r[i, :] = dw_r_i
    all_dw_f[i, :] = dw_f_i
    all_h_v_r[i] = h_v_r_i
    all_h_v_f[i] = h_v_f_i
    all_h_flux_r[i, :] = h_flux_r_i
    all_h_flux_f[i, :] = h_flux_f_i

In [ ]:
spd = 60*60*24
all_dw_r_sum = np.sum((all_dw_r*(all_dw_r>0)).reshape(num_MC,-1),axis = 1)/spd
all_dw_f_sum = np.sum((all_dw_f*(all_dw_f>0)).reshape(num_MC,-1),axis = 1)/spd
all_h_flux_r_sum = np.sum((all_h_flux_r*(all_h_flux_r>0)).reshape(num_MC,-1),axis = 1)/spd
all_h_flux_f_sum = np.sum((all_h_flux_f*(all_h_flux_f>0)).reshape(num_MC,-1),axis = 1)/spd

# Result for Set 2, dry pond

In [ ]:
num_MC = 300

all_dw_r = np.zeros((num_MC, nrow,ncol))
all_dw_f = np.zeros((num_MC, nrow,ncol))
all_h_v_r = np.zeros((num_MC,nlay, nrow))
all_h_v_f = np.zeros((num_MC,nlay, nrow))
all_h_flux_r = np.zeros((num_MC,nlay, nrow))
all_h_flux_f = np.zeros((num_MC,nlay, nrow))

for i in tqdm(range(num_MC)):
    head,flf,frf = read_sim('./Posterior_Simulation_Drypond/sim'+str(i).zfill(3),'sim'+str(i).zfill(3))

    # Calculate responses for current case
    dw_r_i, dw_f_i, h_v_r_i, h_v_f_i, h_flux_r_i, h_flux_f_i = local_regional_response(flf, frf)

    # Store results in arrays
    all_dw_r[i, :] = dw_r_i
    all_dw_f[i, :] = dw_f_i
    all_h_v_r[i] = h_v_r_i
    all_h_v_f[i] = h_v_f_i
    all_h_flux_r[i, :] = h_flux_r_i
    all_h_flux_f[i, :] = h_flux_f_i

In [ ]:
spd = 60*60*24
all_dw_r_sum = np.sum((all_dw_r*(all_dw_r>0)).reshape(num_MC,-1),axis = 1)/spd
all_dw_f_sum = np.sum((all_dw_f*(all_dw_f>0)).reshape(num_MC,-1),axis = 1)/spd
all_h_flux_r_sum = np.sum((all_h_flux_r*(all_h_flux_r>0)).reshape(num_MC,-1),axis = 1)/spd
all_h_flux_f_sum = np.sum((all_h_flux_f*(all_h_flux_f>0)).reshape(num_MC,-1),axis = 1)/spd

# Result for varying: structure, ET, precip, ponding period, K

In [ ]:
parameters = pd.read_csv('./Simulation_More_Variations/Parameters.csv')

In [ ]:
parameters['period'] = pd.factorize(parameters['period'])[0]

In [ ]:
log_paras = parameters[parameters.columns[:-1]]

In [ ]:
log_paras[log_paras.columns[:5]] = np.log10(log_paras[log_paras.columns[:5]])

In [ ]:
def thickness_from_ratio(structure_ratio1,structure_ratio2):
    # setting up the vertical discretization and model bottom elevation
    zbot = np.zeros((nlay,nrow,ncol))
    
    # Soil layers
    for lay in np.arange(0,soil_nlay):    
        zbot[lay,:,:] = DEM - np.maximum(gravel_interface*structure_ratio1*((lay+1)/soil_nlay),0.1*(lay+1)) 
    
    # Gravel layers
    gravel_discretized_ratio = [0.02,0.04,0.1,0.3,0.6,1]
    for i, lay in enumerate(np.arange(soil_nlay, nlay)):
        zbot[lay,:,:] = zbot[soil_nlay-1,:,:] - np.maximum(bedrock_interface*structure_ratio2*gravel_discretized_ratio[i],0.1*(i+1))

    thickness = np.zeros(zbot.shape)
    for i in range(nlay):
        if i == 0:
            thickness[i,:,:] = DEM-zbot[i,:,:]
        else:
            thickness[i,:,:] = zbot[i-1,:,:]-zbot[i,:,:]
    return thickness

In [ ]:
def local_regional_response_structure_change(flf,frf, structure_ratio1,structure_ratio2):
    thickness = thickness_from_ratio(structure_ratio1,structure_ratio2)
    dam_section = 240
    dw_r = dw_river(flf)
    dw_f = dw_floodplain(flf)
    h_v_r = h_dam_river(frf)
    h_v_f = h_dam_floodplain(frf)
    h_flux_r = h_v_r*thickness[:,:,dam_section]
    h_flux_f = h_v_f*thickness[:,:,dam_section]
    return dw_r,dw_f,h_v_r,h_v_f,h_flux_r,h_flux_f

In [ ]:
num_MC = 500

all_dw_r = np.zeros((num_MC, nrow,ncol))
all_dw_f = np.zeros((num_MC, nrow,ncol))
all_h_v_r = np.zeros((num_MC,nlay, nrow))
all_h_v_f = np.zeros((num_MC,nlay, nrow))
all_h_flux_r = np.zeros((num_MC,nlay, nrow))
all_h_flux_f = np.zeros((num_MC,nlay, nrow))
all_head = np.zeros((num_MC,nlay, nrow,ncol))

for i in tqdm(range(num_MC)):
    if parameters['success'][i]==1:
        structure_ratio1 = parameters['structure_ratio1'][i]
        structure_ratio2 = parameters['structure_ratio2'][i]
        head,flf,frf = read_sim('./Simulation_More_Variations/sim'+str(i).zfill(3),'sim'+str(i).zfill(3))
        all_head[i,:]  = head
        # Calculate responses for current case
        dw_r_i, dw_f_i, h_v_r_i, h_v_f_i, h_flux_r_i, h_flux_f_i = local_regional_response_structure_change(flf, frf,structure_ratio1,structure_ratio2)

        # Store results in arrays
        all_dw_r[i, :] = dw_r_i
        all_dw_f[i, :] = dw_f_i
        all_h_v_r[i] = h_v_r_i
        all_h_v_f[i] = h_v_f_i
        all_h_flux_r[i, :] = h_flux_r_i
        all_h_flux_f[i, :] = h_flux_f_i

In [ ]:
spd = 60*60*24 
all_dw_r_sum = np.sum((all_dw_r*(all_dw_r>0)).reshape(num_MC,-1),axis = 1)/spd #(m3/s)
all_dw_f_sum = np.sum((all_dw_f*(all_dw_f>0)).reshape(num_MC,-1),axis = 1)/spd
all_h_flux_r_sum = np.sum((all_h_flux_r*(all_h_flux_r>0)).reshape(num_MC,-1),axis = 1)/spd
all_h_flux_f_sum = np.sum((all_h_flux_f*(all_h_flux_f>0)).reshape(num_MC,-1),axis = 1)/spd

In [ ]:
flux_all = pd.DataFrame(np.array([all_dw_r_sum,all_dw_f_sum,
                                  all_h_flux_r_sum,
                                  all_h_flux_f_sum]).T)

# Define new column names
new_columns = ['$Q_z^r$', '$Q_z^f$', '$Q_x^r$', '$Q_x^f$']

# Rename columns
flux_all.columns = new_columns

In [ ]:
log_paras['period'] = pd.factorize(log_paras['period'])[0]

In [ ]:
import sys

new_path = '../scripts/DGSA_Light/'

sys.path.append(new_path)

from DGSA_light import DGSA_light
from gsa_pareto_plt import gsa_pareto_plt
def colors_from_values(values, palette_name):
    # normalize the values to range [0, 1]
    normalized = (values - 0) / (1 - 0)
    # convert to indices
    indices = np.round(normalized * (len(values) - 1)).astype(np.int32)
    # use the indices to get the colors
    palette = sns.color_palette(palette_name, len(values))
    return np.array(palette).take(indices, axis=0)

In [ ]:
xlabel = ['log $K_{h}^{gravel}$','log $K_{h}^{soil}$',
          'log($K_{v}^{gravel}/K_{h}^{gravel}$)',
          'log($K_{v}^{soil}/K_{h}^{soil}$)','log $K^{dam}$',
          'ET', 'Precipitation','Periods','Soil thickness','Gravel thickness']

In [ ]:
variable = '$Q_z^f$'
dgsa_measures = DGSA_light(log_paras.values[parameters['success']==1,:], flux_all[variable].values.reshape(-1,1)[parameters['success']==1],
                           xlabel)

dgsa_measures['name'] = xlabel
dgsa_measures['sensitive'] = (dgsa_measures[0]>1)*1

dgsa_measures = dgsa_measures.sort_values(by=dgsa_measures.columns[0],ascending=False)

In [ ]:
plt.figure(figsize=(2, 3)) 
sns.barplot(x = 0,y = 'name', data=dgsa_measures,palette = colors_from_values(dgsa_measures['sensitive'], "RdBu_r"))
plt.vlines(x = 1, ymin = -1, ymax = len(log_paras.columns),color = 'black',linestyle = '--')
plt.xlabel('Sensitivity measurement')
plt.ylabel('')
plt.xlim(0,5)
plt.title(variable)

In [ ]:
variable = '$Q_z^f$'
dgsa_measures = DGSA_light(log_paras.values[parameters['success']==1,:], 
                           flux_all[variable].values.reshape(-1,1)[parameters['success']==1]/flux_all['$Q_x^f$'].values.reshape(-1,1)[parameters['success']==1],
                           xlabel)

dgsa_measures['name'] = xlabel
dgsa_measures['sensitive'] = (dgsa_measures[0]>1)*1

dgsa_measures = dgsa_measures.sort_values(by=dgsa_measures.columns[0],ascending=False)

In [ ]:
plt.figure(figsize=(2, 3)) 
sns.barplot(x = 0,y = 'name', data=dgsa_measures,palette = colors_from_values(dgsa_measures['sensitive'], "RdBu_r"))
plt.vlines(x = 1, ymin = -1, ymax = len(log_paras.columns),color = 'black',linestyle = '--')
plt.xlabel('Sensitivity measurement')
plt.ylabel('')
plt.xlim(0,5)
plt.title(variable+'/$Q_x^f$')

In [ ]:
log_paras = log_paras.drop(columns=['period'])

In [ ]:
xlabel = ['log $K_{h}^{gravel}$','log $K_{h}^{soil}$',
          'log($K_{v}^{gravel}/K_{h}^{gravel}$)',
          'log($K_{v}^{soil}/K_{h}^{soil}$)','log $K^{dam}$',
          'ET', 'Precipitation','Soil thickness','Gravel thickness']

In [ ]:
all_dw_f_m_s = all_dw_f/spd

In [ ]:
i = -4
plt.figure(figsize=[5,5])
pond_vis = np.array(np.copy(pond),dtype = 'float64')
pond_vis[pond_vis ==0] = np.nan
SA_matrix_vis = SA_matrix[i,:,:]
SA_matrix_vis[SA_matrix_vis<1] = np.nan
plt.imshow(SA_matrix_vis,vmin = 1,vmax = 3.8,cmap = 'PuRd')
plt.colorbar(shrink = 0.3)
plt.imshow(pond_vis,cmap = 'Blues_r',alpha = 0.1)
plt.axis('off')
plt.title(xlabel[i])

In [ ]:
i = 1
plt.figure(figsize=[5,5])
pond_vis = np.array(np.copy(pond),dtype = 'float64')
pond_vis[pond_vis ==0] = np.nan
SA_matrix_vis = SA_matrix[i,:,:]
SA_matrix_vis[SA_matrix_vis<1] = np.nan
plt.imshow(SA_matrix_vis,vmin = 1,vmax = 3.8,cmap = 'PuRd')
plt.colorbar(shrink = 0.3)
plt.imshow(pond_vis,cmap = 'Blues_r',alpha = 0.1)
plt.axis('off')
plt.title(xlabel[i])

In [ ]:
i = 3
plt.figure(figsize=[5,5])
pond_vis = np.array(np.copy(pond),dtype = 'float64')
pond_vis[pond_vis ==0] = np.nan
SA_matrix_vis = SA_matrix[i,:,:]
SA_matrix_vis[SA_matrix_vis<1] = np.nan
plt.imshow(SA_matrix_vis,vmin = 1,vmax = 3.8,cmap = 'PuRd')
plt.colorbar(shrink = 0.3)
plt.imshow(pond_vis,cmap = 'Blues_r',alpha = 0.1)
plt.axis('off')
plt.title(xlabel[i])

In [ ]:
parameters

In [ ]:
plt.scatter(parameters['structure_ratio1'],
            all_dw_f_sum/all_h_flux_f_sum,c = np.log10(parameters['hk_soil']*parameters['vka_ratio_soil']/parameters['hk_gravel']))
plt.colorbar()
plt.yscale('log') 
#plt.xscale('log') 

In [ ]:
gravel_interface = np.load('../model/subsurface/predict_gravel.npy')
gravel_interface[np.isnan(gravel_interface)] = 0
bedrock_interface = np.load('../model/subsurface/predict_bedrock.npy')
bedrock_interface[np.isnan(bedrock_interface)] = 0


In [ ]:
from matplotlib.ticker import FuncFormatter

# Assuming `parameters`, `all_dw_f_sum`, and `all_h_flux_f_sum` are already defined

# Filter out period == 1
filtered_parameters = parameters[parameters['period'] != 1]
structure_ratio2 = filtered_parameters['structure_ratio2']
hk_soil = filtered_parameters['hk_soil']
vka_ratio_soil = filtered_parameters['vka_ratio_soil']
hk_gravel = filtered_parameters['hk_gravel']

# Calculate the color array
color = np.log10(hk_soil * vka_ratio_soil )

# Calculate the y values for the scatter plot
y_values = (all_dw_f_sum / all_h_flux_f_sum)[parameters['period'] != 1]

# Create the scatter plot
plt.scatter(structure_ratio2, y_values,s = 20, c=color,vmin = -7.5,vmax = -4.5)
plt.colorbar(label='$\log(K_{v}^{soil})$')
plt.xlim(0.01,3)
plt.yscale('log') 
plt.xscale('log') 
plt.xlabel('Gravel Thickness (m)')
plt.ylabel('$\log(Q_z^f/Q_x^f)$')

# Define the custom formatter function
def custom_xaxis_formatter(x, pos):
    if x == 0.1:
        return '1.4'
    elif x == 1:
        return '14'
    elif x == 0.01:
        return '0.14'
    else:
        return f'{x}'

# Apply the custom formatter to the x-axis
formatter = FuncFormatter(custom_xaxis_formatter)
plt.gca().xaxis.set_major_formatter(formatter)

# Show the plot
plt.show()


In [ ]:
from matplotlib.ticker import FuncFormatter

# Assuming `parameters`, `all_dw_f_sum`, and `all_h_flux_f_sum` are already defined

# Filter out period == 1
filtered_parameters = parameters[parameters['period'] != 1]
structure_ratio2 = filtered_parameters['structure_ratio2']
hk_soil = filtered_parameters['hk_soil']
vka_ratio_soil = filtered_parameters['vka_ratio_soil']
hk_gravel = filtered_parameters['hk_gravel']

# Calculate the color array
color = np.log10(hk_soil * vka_ratio_soil / hk_gravel)

# Calculate the y values for the scatter plot
y_values = (all_dw_f_sum / all_h_flux_f_sum)[parameters['period'] != 1]

# Create the scatter plot
plt.scatter(structure_ratio2, y_values, c=color)
plt.colorbar(label='$\log(K_{v}^{soil}/K_{h}^{gravel})$')
plt.xlim(0.01,3)
plt.yscale('log') 
plt.xscale('log') 
plt.xlabel('Gravel Thickness (m)')
plt.ylabel('$\log(Q_z^f/Q_x^f)$')

# Define the custom formatter function
def custom_xaxis_formatter(x, pos):
    if x == 0.1:
        return '1.4'
    elif x == 1:
        return '14'
    elif x == 0.01:
        return '0.14'
    else:
        return f'{x}'

# Apply the custom formatter to the x-axis
formatter = FuncFormatter(custom_xaxis_formatter)
plt.gca().xaxis.set_major_formatter(formatter)

# Show the plot
plt.show()


In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm

# Define colors and labels for each category
colors = ['C3', 'C8', 'C0']  # Corresponding to 2: 'Snowmelt', 1: 'Dry Pond', 0: 'Baseflow'
labels = {0: 'Baseflow',1: 'Dry Pond', 2: 'Snowmelt' }
cmap = ListedColormap(colors)
bounds = [0, 1, 2, 3]  # Set boundaries for each category
norm = BoundaryNorm(bounds, cmap.N)

# Create the scatter plot
scatter = plt.scatter(parameters['ET'], all_dw_f_sum, c=parameters['period'], cmap=cmap, norm=norm)
plt.yscale('log')
plt.xlabel('ET [m/d]')
plt.ylabel('$\log(Q_z^f) \ [m^3/s]$')

# Create a color bar with appropriate labels for the periods
cbar = plt.colorbar(scatter, ticks=[0.5, 1.5, 2.5])  # Position the ticks between boundaries
cbar.ax.set_yticklabels([labels[0], labels[1], labels[2]])  # Apply labels

# Show the plot
plt.show()


In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm

# Define colors and labels for each category
colors = ['C3', 'C8', 'C0']  # Corresponding to 2: 'Snowmelt', 1: 'Dry Pond', 0: 'Baseflow'
labels = {0: 'Baseflow',1: 'Dry Pond', 2: 'Snowmelt' }
cmap = ListedColormap(colors)
bounds = [0, 1, 2, 3]  # Set boundaries for each category
norm = BoundaryNorm(bounds, cmap.N)

# Create the scatter plot
scatter = plt.scatter(parameters['ET'][parameters['period']!=1], all_dw_f_sum[parameters['period']!=1], c=parameters['period'][parameters['period']!=1], cmap=cmap, norm=norm)
plt.yscale('log')
plt.xlabel('ET [m/d]')
plt.ylabel('$\log(Q_z^f) \ [m^3/s]$')

# Create a color bar with appropriate labels for the periods
cbar = plt.colorbar(scatter, ticks=[0.5, 1.5, 2.5])  # Position the ticks between boundaries
cbar.ax.set_yticklabels([labels[0], labels[1], labels[2]])  # Apply labels

# Show the plot
plt.show()


In [ ]:
plt.scatter(parameters['ET'],all_dw_f_sum, c= parameters['period']) # 2=C0, 1=C8, 0=C3
plt.yscale('log')
plt.xlabel('ET [m/d]')
plt.ylabel('$log(Q_z^f) [m3/s]$')
plt.colorbar()

In [ ]:
plt.scatter(parameters['ET'],all_dw_f_sum*86400, c= parameters['period'])
plt.yscale('log')
plt.xlabel('ET [m/d]')
plt.ylabel('$log(Q_z^f) [m3/s]$')
plt.colorbar()

In [ ]:
pond_floodplain = ((pond+downstream+upstream)==2)*1

In [ ]:
plt.imshow(all_dw_f[0,:,:]-0.005*pond_floodplain,vmin = -0.01,vmax = 0.01,cmap = 'RdBu') #np.where(parameters['period']==0)[0][0],:,:],
plt.colorbar()

In [ ]:
pond = np.load('../data/characterization_data/Beaver_pond_dam/pond_baseflow.npy')
pond_sm = np.load('../data/characterization_data/Beaver_pond_dam/pond_snowmelt.npy')
   

In [ ]:
pond_floodplain = ((pond+downstream+upstream)==2)*1
pond_floodplain_sm = ((pond_sm+downstream+upstream)==2)*1

In [ ]:
correction_ET = np.zeros(500)
for i in range(500):
    if parameters['period'][i]==0:
        correction_ET[i] = np.sum(parameters['ET'][i]*pond_floodplain)/86400
    elif parameters['period'][i]==2:
        correction_ET[i] = np.sum(parameters['ET'][i]*pond_floodplain_sm)/86400

In [ ]:
all_dw_f_corrected = np.zeros(all_dw_f.shape)
for i in range(500):
    if parameters['period'][i]==0:
        all_dw_f_corrected[i,:,:] = all_dw_f[i,:,:]-parameters['ET'][i]*pond_floodplain
    elif parameters['period'][i]==2:
        all_dw_f_corrected[i,:,:] = all_dw_f[i,:,:]-parameters['ET'][i]*pond_floodplain_sm

In [ ]:
all_dw_f_corrected_sum = np.sum((all_dw_f_corrected*(all_dw_f_corrected>0)).reshape(num_MC,-1),axis = 1)/spd

In [ ]:
data = {
    'corrected_Qzf': all_dw_f_corrected_sum,
    'Qzf': all_dw_f_sum
}

# Convert the arrays to a DataFrame
arrays_df = pd.DataFrame(data)

# Concatenate the DataFrame with parameters, ensuring correct alignment
combined_df = pd.concat([arrays_df, parameters], axis=1)
df =combined_df

In [ ]:
df =combined_df

In [ ]:
df_1_1_Qzf = df[(df['period'] == 1) & (df['success'] == 1)]['Qzf']
df_0_1_Qzf = df[(df['period'] == 0) & (df['success'] == 1)]['Qzf']
df_0_1_corrected_Qzf = df[(df['period'] == 0) & (df['success'] == 1)]['corrected_Qzf']
df_2_1_Qzf = df[(df['period'] == 2) & (df['success'] == 1)]['Qzf']
df_2_1_corrected_Qzf = df[(df['period'] == 2) & (df['success'] == 1)]['corrected_Qzf']

In [ ]:
# Combine data into a list for the boxplot
data_to_plot = [df_1_1_Qzf, df_0_1_Qzf, df_0_1_corrected_Qzf, df_2_1_Qzf, df_2_1_corrected_Qzf]

# Create boxplots based on the conditions
fig, ax = plt.subplots(figsize=(10, 6))
# Create boxplot
ax.boxplot(data_to_plot, labels=[
    'Qzf (Period 1, Success 1)', 
    'Qzf (Period 0, Success 1)', 
    'Corrected Qzf (Period 0, Success 1)', 
    'Qzf (Period 2, Success 1)', 
    'Corrected Qzf (Period 2, Success 1)'
])

# Set labels and title
ax.set_ylabel('Values')
ax.set_title('Boxplots for Qzf and Corrected Qzf by Period and Success')

# Display the plot
plt.xticks(rotation=45)
plt.tight_layout()
plt.yscale('log')
plt.show()

In [ ]:
import numpy as np

# Fit linear lines for each period and plot them
# For Baseflow (period = 0)
baseflow_x = df[(df['period'] == 0) & (df['success'] == 1)]['ET']
baseflow_y = df[(df['period'] == 0) & (df['success'] == 1)]['corrected_Qzf']
baseflow_fit = np.polyfit(baseflow_x, np.log(baseflow_y), 1)  # Log scale fit
plt.plot(baseflow_x, np.exp(np.polyval(baseflow_fit, baseflow_x)), 'C3',linestyle = '--', label='Baseflow fit')


baseflow_x = df[(df['period'] == 0) & (df['success'] == 1)]['ET']
baseflow_y = df[(df['period'] == 0) & (df['success'] == 1)]['Qzf']
baseflow_fit = np.polyfit(baseflow_x, np.log(baseflow_y), 1)  # Log scale fit
plt.plot(baseflow_x, np.exp(np.polyval(baseflow_fit, baseflow_x)), 'C3', label='Baseflow fit')


# For Dry Pond (period = 1)
dry_pond_x = df[(df['period'] == 1) & (df['success'] == 1)]['ET']
dry_pond_y = df[(df['period'] == 1) & (df['success'] == 1)]['Qzf']
dry_pond_fit = np.polyfit(dry_pond_x, np.log(dry_pond_y), 1)  # Log scale fit
plt.plot(dry_pond_x, np.exp(np.polyval(dry_pond_fit, dry_pond_x)), 'C8', label='Dry Pond fit')


# For Snowmelt (period = 2)
snowmelt_x = df[(df['period'] == 2) & (df['success'] == 1)]['ET']
snowmelt_y = df[(df['period'] == 2) & (df['success'] == 1)]['Qzf']
snowmelt_fit = np.polyfit(snowmelt_x, np.log(snowmelt_y), 1)  # Log scale fit
plt.plot(snowmelt_x, np.exp(np.polyval(snowmelt_fit, snowmelt_x)), 'C0', label='Snowmelt fit')


# For Snowmelt (period = 2)
snowmelt_x = df[(df['period'] == 2) & (df['success'] == 1)]['ET']
snowmelt_y = df[(df['period'] == 2) & (df['success'] == 1)]['corrected_Qzf']
snowmelt_fit = np.polyfit(snowmelt_x, np.log(snowmelt_y), 1)  # Log scale fit
plt.plot(snowmelt_x, np.exp(np.polyval(snowmelt_fit, snowmelt_x)), 'C0',linestyle = '--', label='Snowmelt fit')

# Scatter plots (with fitted lines)
#plt.scatter(baseflow_x, baseflow_y, edgecolors='C3', c='C3', marker='^', label='Baseflow')
#plt.scatter(dry_pond_x, dry_pond_y, edgecolors='C8', c='C8', marker='o', label='Dry Pond')
#plt.scatter(snowmelt_x, snowmelt_y, edgecolors='C0', c='C0', marker='s', label='Snowmelt')

# Adding labels and color bar as before
plt.yscale('log')
plt.xlabel('ET [m/d]')
plt.ylabel('$\log(Q_z^f) \ [m^3/s]$')

# Re-add color bar for the scatter plot
cbar = plt.colorbar(scatter, ticks=[0.5, 1.5, 2.5])
cbar.ax.set_yticklabels([labels[0], labels[1], labels[2]])

# Show the plot with legends for the lines
plt.legend()
plt.show()


In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm

# Define colors and labels for each category
colors = ['C3', 'C8', 'C0']  # Corresponding to 2: 'Snowmelt', 1: 'Dry Pond', 0: 'Baseflow'
labels = {0: 'Baseflow',1: 'Dry Pond', 2: 'Snowmelt' }
cmap = ListedColormap(colors)
bounds = [0, 1, 2, 3]  # Set boundaries for each category
norm = BoundaryNorm(bounds, cmap.N)


# Create the scatter plot
scatter = plt.scatter(parameters['ET'][(parameters['success']==1)&(parameters['period']==1)], all_dw_f_sum[(parameters['success']==1)&(parameters['period']==1)], c = parameters['period'][(parameters['success']==1)&(parameters['period']==1)], cmap=cmap, norm=norm)
plt.yscale('log')
plt.xlabel('ET [m/d]')
plt.ylabel('$\log(Q_z^f) \ [m^3/s]$')

plt.scatter(df[(df['period'] == 2) & (df['success'] == 1) & (df['zf_change'] >0.2)]['ET'],df[(df['period'] == 2) & (df['success'] == 1) & (df['zf_change'] >0.2)]['corrected_Qzf'],marker = '^',c='C0')
plt.scatter(df[(df['period'] == 0) & (df['success'] == 1) & (df['zf_change'] >0.2)]['ET'],df[(df['period'] == 0) & (df['success'] == 1) & (df['zf_change'] >0.2)]['corrected_Qzf'],marker = '^',c='C3')

# Create a color bar with appropriate labels for the periods
cbar = plt.colorbar(scatter, ticks=[0.5, 1.5, 2.5])  # Position the ticks between boundaries
cbar.ax.set_yticklabels([labels[0], labels[1], labels[2]])  # Apply labels

#plt.plot(df[(df['period'] == 1) & (df['success'] == 1)]['ET'],df[(df['period'] == 1) & (df['success'] == 1)]['Qzf'],'.')
plt.yscale('log')

# Show the plot
plt.show()



In [ ]:
from matplotlib.colors import ListedColormap, BoundaryNorm

# Define colors and labels for each category
colors = ['C3', 'C8', 'C0']  # Corresponding to 2: 'Snowmelt', 1: 'Dry Pond', 0: 'Baseflow'
labels = {0: 'Baseflow',1: 'Dry Pond', 2: 'Snowmelt' }
cmap = ListedColormap(colors)
bounds = [0, 1, 2, 3]  # Set boundaries for each category
norm = BoundaryNorm(bounds, cmap.N)


# Create the scatter plot
scatter = plt.scatter(parameters['ET'][(parameters['success']==1)&(parameters['period']>-1)], all_dw_f_sum[(parameters['success']==1)&(parameters['period']>-1)], c = parameters['period'][(parameters['success']==1)&(parameters['period']>-1)], cmap=cmap, norm=norm)
plt.yscale('log')
plt.xlabel('ET [m/d]')
plt.ylabel('$\log(Q_z^f) \ [m^3/s]$')

plt.scatter(df[(df['period'] == 2) & (df['success'] == 1) ]['ET'],df[(df['period'] == 2) & (df['success'] == 1)]['Qzf'],c =  'C0',label = 'MC Set 3')

plt.scatter(df[(df['period'] == 2) & (df['success'] == 1) ]['ET'],df[(df['period'] == 2) & (df['success'] == 1)]['corrected_Qzf'],edgecolors='C0',c = 'white',label = 'ET_Corrected \nMC Set3')
plt.scatter(df[(df['period'] == 0) & (df['success'] == 1) ]['ET'],df[(df['period'] == 0) & (df['success'] == 1) ]['corrected_Qzf'],edgecolors='C3',c = 'white')

plt.legend()
# Create a color bar with appropriate labels for the periods
cbar = plt.colorbar(scatter, ticks=[0.5, 1.5, 2.5])  # Position the ticks between boundaries
cbar.ax.set_yticklabels([labels[0], labels[1], labels[2]])  # Apply labels

#plt.plot(df[(df['period'] == 1) & (df['success'] == 1)]['ET'],df[(df['period'] == 1) & (df['success'] == 1)]['Qzf'],'.')
plt.yscale('log')

plt.title('ET Corrected $Q_z^f$')

# Show the plot
plt.show()



In [ ]:
df['zf_change'] = (df['Qzf']-df['corrected_Qzf'])/df['Qzf']

In [ ]:
scatter = plt.scatter(df[(df['period'] == 2) & (df['success'] == 1)]['hk_soil']*df[(df['period'] == 2) & (df['success'] == 1)]['vka_ratio_soil'],
            df[(df['period'] == 2) & (df['success'] == 1)]['structure_ratio1']*2,c = np.log10(df[(df['period'] == 2) & (df['success'] == 1)]['corrected_Qzf']),vmin = -6,vmax= -2,cmap = 'RdBu')
plt.xlabel('$log(K_v^{soil})$')
plt.ylabel('Soil thickness (m)')
#plt.yscale('log')
plt.xscale('log')
# Adding color bar with title
cbar = plt.colorbar(scatter)
cbar.set_label('$\log($ET-corrected $ Q_z^f)$')

plt.title('Simulations for Snowmelt Period from MC Set 3')


In [ ]:
plt.scatter(df[(df['period'] == 0) & (df['success'] == 1)]['hk_soil']*df[(df['period'] == 0) & (df['success'] == 1)]['vka_ratio_soil'],
            df[(df['period'] == 0) & (df['success'] == 1)]['structure_ratio1'],c = np.log10(df[(df['period'] == 0) & (df['success'] == 1)]['corrected_Qzf']),vmin = -7,vmax= -2)

#plt.yscale('log')
plt.xscale('log')
plt.colorbar()

In [ ]:
df[(df['period'] == 2) & (df['success'] == 1) & (df['corrected_Qzf'] <1e-4) ]

In [ ]:
df[(df['period'] == 2) & (df['success'] == 1) & (df['corrected_Qzf'] >1e-4) ]

In [ ]:
plt.hist(all_dw_f_corrected_sum[parameters['success']==1]/all_dw_f_sum[parameters['success']==1])


In [ ]:
all_dw_f_corrected_sum

In [ ]:
np.sum(pond_floodplain*ET)/86400

In [ ]:
np.sum(pond_floodplain_sm*ET)/86400

In [ ]:
all_dw_f_sum[1]

In [ ]:
correction_ET[1]

In [ ]:
parameters.iloc[1].values

In [ ]:
correction_ET[parameters['success']==0] = 0
all_dw_f_sum[parameters['success']==0] = 100

In [ ]:
np.argmax((correction_ET/all_dw_f_sum))

In [ ]:
all_dw_f_sum[382]

In [ ]:
correction_ET[382]

In [ ]:
plt.imshow(all_dw_f[382,:,:],vmin = -0.01,vmax = 0.01,cmap = 'RdBu')

In [ ]:
parameters.iloc[382]

In [ ]:
all_dw_f_sum[320]

In [ ]:
correction_ET[320]/all_dw_f_sum[320]

In [ ]:
np.sum((all_dw_f[0,:,:]-parameters['ET'][0]*pond_floodplain)[(all_dw_f[0,:,:]-0.005*pond_floodplain)>0])/(86400)

In [ ]:
all_dw_f[periods]

In [ ]:
plt.imshow(np.mean(all_dw_f[parameters['period']==0,:,:],axis = 0)>0.005)

In [ ]:
plt.scatter(parameters['ET'],parameters['precip'],c = np.log10(all_dw_f_sum))
#plt.yscale('log')
plt.colorbar()

In [ ]:
from matplotlib.ticker import FuncFormatter

# Assuming `parameters`, `all_dw_f_sum`, and `all_h_flux_f_sum` are already defined

# Filter out period == 1
filtered_parameters = parameters[parameters['period'] != 1]
structure_ratio2 = filtered_parameters['structure_ratio2']
hk_soil = filtered_parameters['hk_soil']
vka_ratio_soil = filtered_parameters['vka_ratio_soil']
hk_gravel = filtered_parameters['hk_gravel']

# Calculate the color array
color = np.log10(hk_soil * vka_ratio_soil / hk_gravel)

# Calculate the y values for the scatter plot
y_values = (all_dw_f_sum / all_h_flux_f_sum)[parameters['period'] != 1]

# Create the scatter plot
plt.scatter(structure_ratio2, y_values, c=color)
plt.colorbar(label='$\log(K_{v}^{soil}/K_{h}^{gravel})$')
plt.xlim(0.01,3)
plt.yscale('log') 
plt.xscale('log') 
plt.xlabel('Gravel Thickness (m)')
plt.ylabel('$\log(Q_z^f/Q_x^f)$')

# Define the custom formatter function
def custom_xaxis_formatter(x, pos):
    if x == 0.1:
        return '1.4'
    elif x == 1:
        return '14'
    elif x == 0.01:
        return '0.14'
    else:
        return f'{x}'

# Apply the custom formatter to the x-axis
formatter = FuncFormatter(custom_xaxis_formatter)
plt.gca().xaxis.set_major_formatter(formatter)

# Show the plot
plt.show()


In [ ]:
SA_matrix = np.zeros((len(xlabel), nrow, ncol))
for i in range(nrow):
    print(i)
    for j in range(ncol):
        if (upstream+downstream)[i,j]!=0:
            criterion = (parameters['period']=='baseflow') & (parameters['success']==1)
            max_attempts = 10
            attempts = 0
            success = False
            while attempts < max_attempts and not success:
                try:
                    dgsa_measures = DGSA_light(log_paras.values[criterion,:], 
                                       all_dw_f[criterion,i,j].reshape(-1,1),
                                       xlabel,n_clsters=3,n_boots=3000)
                    success = True
                except ValueError as e:
                    attempts += 1
                    print(f"Attempt {attempts} failed: {e}")
            SA_matrix[:,i,j] = dgsa_measures[0].values

In [ ]:
np.save('SA_baseflow.npy',SA_matrix)

In [ ]:
dgsa_measures[0].values

In [ ]:
ET_SA_baseflow = 
plt.imshow(upstream+downstream)

In [ ]:
precip_SA_baseflow = 
plt.imshow(upstream+downstream)

In [ ]:
hk_soil_SA_baseflow = 
plt.imshow(upstream+downstream)